# Reducción de la dimensionalidad

En muchos problemas de aprendizaje automático, especialmente aquellos con un gran número de variables o características, es fundamental reducir la dimensionalidad del conjunto de datos.  
Este proceso permite simplificar el modelo, mejorar la interpretabilidad y, en muchos casos, aumentar el rendimiento y la eficiencia del aprendizaje.

En este notebook se ilustran distintas técnicas de reducción de dimensionalidad implementadas en Julia, utilizando principalmente el framework **MLJ**, que ofrece una interfaz unificada para el entrenamiento y evaluación de modelos.

## Instalación
Antes de comenzar, se recomienda verificar que todas las dependencias necesarias estén correctamente instaladas en el entorno de trabajo.


In [ ]:
using Pkg
Pkg.add(["MLJ",
        "MLJModels",
        "MLJModelInterface",
        "MultivariateStats",
        "HypothesisTests",
        "RDatasets",
        "DataFrames",
        "CategoricalArrays",
        "StatsBase",
        "Plots"])

## Importación de dependencias

Este notebook no pretende ser un ejemplo exhaustivo de todas las posibles técnicas de reducción de dimensionalidad, sino una guía práctica centrada en el uso de **MLJ** y algunas de sus extensiones más comunes.  

Cabe destacar que ciertas técnicas, como *Isomap* o *Locally Linear Embedding (LLE)*, no están aún integradas directamente en MLJ y requieren la creación de *wrappers* específicos para su uso dentro de este framework. Esto ya fue cubierto en un notebook anterior y se le deja a los y las estudiantes para que lo exploren

A continuación, se importan las librerías y módulos necesarios para ejecutar los ejemplos.


In [ ]:
using MLJ
using MLJBase
using MLJTransforms
using MLJModelInterface
using RDatasets
using MultivariateStats
using HypothesisTests
using DataFrames
using CategoricalArrays
using StatsBase
using Plots

## Conjunto de datos: Iris

Como ejemplo práctico, utilizaremos el clásico conjunto de datos **Iris**, ampliamente empleado en tareas de clasificación y reducción de dimensionalidad.  

Dividiremos el conjunto en dos partes: un conjunto de **entrenamiento** y otro de **prueba**, con el fin de evaluar el comportamiento de las diferentes técnicas de reducción en datos no vistos.


In [ ]:
# Cargar el dataset Iris
iris = dataset("datasets", "iris")
y, X = unpack(iris, ==(:Species)); ## Un vector y una tabla resultante

# === División en entrenamiento y prueba ===
# 70% entrenamiento, 30% test, con semilla reproducible
idx_train, idx_test = partition(eachindex(y), 0.7, rng=42); 

(X_train, X_test), (y_train, y_test) = (X[idx_train, :], X[idx_test, :]), (y[idx_train], y[idx_test]);

@info "Train dataset $(size(X_train)) -> $(length(y_train))"
@info "Test dataset  $(size(X_test)) -> $(length(y_test))"

#### Definición de las técnicas de reducción

En esta sección definiremos las principales técnicas de reducción que se van a emplear.  
Nos centraremos en aquellas disponibles en las librerías compatibles con MLJ —como PCA, ICA y LDA— aunque es posible incorporar muchas otras.

Las técnicas de tipo *manifold learning*, como **Isomap** o **LLE**, son transformaciones no lineales útiles para la representación y visualización de datos en espacios de menor dimensión, si bien no siempre permiten una reconstrucción inversa.

In [ ]:
# Primero es necesario cargarlas librerías
PCA = @load PCA pkg=MultivariateStats add=true
ICA = @load ICA pkg=MultivariateStats
LDA = @load LDA pkg=MultivariateStats


# Definimos un diccionario con las técnicas de reducción de dimensionalidad
techniques=Dict(
    "PCA" => PCA(maxoutdim=2),
    "ICA" => ICA(outdim=2, maxiter=500, tol=1e-1, do_whiten=true),
    "LDA" => LDA(outdim=2)
)


## Filtrado de características

Además de las técnicas de proyección, es posible aplicar **métodos de filtrado** para la selección de características más relevantes.  
Entre ellos destacan los basados en medidas estadísticas como *Pearson*, *ANOVA* o *Spearman*.  

En este ejemplo se mostrará cómo integrar un filtro ANOVA dentro del flujo de trabajo, empleando código similar al visto en el *Tutorial 2*.  
Estos métodos resultan útiles para eliminar atributos redundantes antes de aplicar técnicas de reducción más complejas.


In [ ]:
using HypothesisTests

# ===============================
# ANOVA SelectKBest Filter
# ===============================
# Definir la estructura del modelo
mutable struct ANOVAFilter <: MLJModelInterface.Supervised
    k::Int
end

# Constructor con argumentos por nombre (estilo MLJ)
ANOVAFilter(; k::Int=2) = ANOVAFilter(k)

# Se hace la importación de las funciones necesarias para poder sobrecargarlas
import MLJModelInterface: fit, transform, input_scitype, target_scitype, output_scitype

# Definir la función fit
function fit(model::ANOVAFilter, verbosity::Int, X, y)
    # Convertir X a matriz y asegurarse que es Float64
    Xmat = Float64.(MLJBase.matrix(X))
    
    # Convertir y a vector numérico si es categórico
    y_numeric = y isa CategoricalVector ? Int.(levelcode.(y)) : Int.(y)
    
    # Calcular F-statistics usando HypothesisTests
    n_features = size(Xmat, 2)
    fstats = zeros(Float64, n_features)
    
    for j in 1:n_features
        feature = Xmat[:, j]
        
        # Agrupar datos por clase
        classes = unique(y_numeric)
        groups = [feature[y_numeric .== c] for c in classes]
        
        # Filtrar grupos vacíos
        groups = filter(g -> length(g) > 0, groups)
        
        if length(groups) < 2
            fstats[j] = 0.0
            continue
        end
        
        try
            # Crear test ANOVA
            test = OneWayANOVATest(groups...)
            
            # Calcular F-statistic manualmente desde los campos disponibles
            MSt = test.SStᵢ / test.DFt  # Mean square treatment (between)
            MSe = test.SSeᵢ / test.DFe  # Mean square error (within)
            
            fstats[j] = MSt / MSe
        catch e
            # Si hay algún error en el cálculo, asignar 0
            fstats[j] = 0.0
        end
    end
    
    # Seleccionar top k features con mayor F-statistic
    k_actual = min(model.k, n_features)
    idxs = sortperm(fstats, rev=true)[1:k_actual]
    
    # Guardar los nombres de las columnas originales
    feature_names = collect(Tables.columnnames(X))
    selected_names = [feature_names[i] for i in idxs]
    
    # IMPORTANTE: fitresult debe contener la info necesaria para transform
    fitresult = (idxs=idxs, selected_names=selected_names)
    cache = nothing
    report = (fstats=fstats, idxs=idxs, selected_features=selected_names)
    
    return fitresult, cache, report
end

# Definir la función transform
function transform(model::ANOVAFilter, fitresult, X)
    # Convertir X a matriz
    Xmat = MLJBase.matrix(X)
    
    # Seleccionar columnas usando fitresult (no cache)
    X_selected = Xmat[:, fitresult.idxs]
    
    # Convertir de vuelta a tabla con nombres apropiados
    return MLJBase.table(X_selected, names=fitresult.selected_names)
end

# Definir los tipos de entrada y salida
input_scitype(::Type{<:ANOVAFilter}) = Table(Continuous)
target_scitype(::Type{<:ANOVAFilter}) = AbstractVector{<:Finite}
output_scitype(::Type{<:ANOVAFilter}) = Table(Continuous)


techniques["Filtrado (ANOVA)"] = ANOVAFilter(k=2)

## Construcción de *Pipelines*

Una vez definidas las distintas técnicas, el siguiente paso consiste en utilizarlas para construir *pipelines* que combinen varias etapas del flujo de trabajo: estandarización, reducción de dimensionalidad y clasificación.  

Esto permitirá comparar fácilmente el rendimiento de cada combinación y observar las diferencias entre los distintos enfoques.

Un elemento clave en este punto es que se están mezclando dos tipos de técnicas, supervisadas y no supervisadas, como transformadores. Esto de primeras si se usa la clase *Pipeline* dará un error, por lo que será necesario hacer el pipeline en dos etapas. Si no se usasen los transformadores LDA e ANOVA se podría integrar todo en un *pipeline*

In [ ]:
using MLJ, DataFrames, RDatasets, Statistics

# Cargar modelo clasificición y estandarizador
Standardizer = @load Standardizer pkg=MLJModels
LogisticClassifier = @load LogisticClassifier pkg=MLJLinearModels


# Técnicas que viene definidas de las celdas anteriores
#techniques = Dict(
#    "PCA" => PCA(maxoutdim=2),
#    "ICA" => ICA(outdim=2, maxiter=500, tol=1e-1, do_whiten=true),
#    "LDA" => LDA(outdim=2),
#    "ANOVA_k2" => ANOVAFilter(k=2)
#)

# Definidión Resultados
results = DataFrame(Technique=String[], Accuracy=Float64[])

# Estandarizar los datos
std_model = Standardizer()
std_mach = machine(std_model, X_train)
fit!(std_mach)
Xs_train = transform(std_mach, X_train)
Xs_test  = transform(std_mach, X_test)

machines = Dict()

# Comprobar cada técnica de reducción
for (name, reducer) in techniques
    println("\nEntrenando con: $name (dos etapas)")
    try
        # --- Etapa 1: Reducción --- 
        
        if MLJModelInterface.is_supervised(reducer)
            # Caso supervisado en el que se usa X e y
            machines[name] = machine(reducer, Xs_train, y_train)
        else
            # Caso no supervisado en el que se usa solo X
            machines[name] = machine(reducer, Xs_train)
        end
        fit!(machines[name])
        # Realizar las transformaciones
        Xredudeced_train = transform(machines[name], Xs_train)
        Xreduced_test  = transform(machines[name], Xs_test)

        # --- Etapa 2: Clasificador ---
        clf = LogisticClassifier()
        clf_mach = machine(clf, Xredudeced_train, y_train)
        fit!(clf_mach)
        y_pred = predict_mode(clf_mach, Xreduced_test)

        acc = mean(y_pred .== y_test)
        push!(results, (name, acc))
        println("✓ Precisión: ", round(acc, digits=4))

    catch e
        println("✗ Error en $name: $e")
    end
end

# Imprimir los resultados para todos los métodos probados
sort!(results, :Accuracy, rev=true)
println("\n", "="^50)
println("RESULTADOS FINALES")
println("="^50)
for row in eachrow(results)
    println("$(rpad(row.Technique, 15)) → $(round(row.Accuracy * 100, digits=2))%")
end

Una vez definido un diccionario con dichas técnicas, el siguiente paso es usarlo para definir diferentes *Pipelines* y comprobar las diferencias entre ellos.

# Representación visual

Por último, representaremos los datos transformados por cada una de las técnicas de reducción de dimensionalidad.  
Para ello haremos uso de una función auxiliar y del paquete **Plots**, que nos permitirá generar gráficos bidimensionales donde se puedan apreciar las separaciones entre clases tras la transformación del espacio original.


In [ ]:
using Plots

# Generar scatterplots de las transformaciones
function plot_transformed_data(name::String, reducer, X, y)
    # Ajustar reductor
    if MLJModelInterface.is_supervised(reducer)
        mach_red = machine(reducer, Xs, y)
    else
        mach_red = machine(reducer, Xs)
    end
    fit!(mach_red, verbosity=0)

    X_reduced = transform(mach, X)
    
    # Crear el scatterplot
    p = scatter(X_reduced[:, 1], X_reduced[:, 2], group=y, legend=:topright, title=name,
                xlabel="Componente 1", ylabel="Componente 2", markersize=5)
    return p
end

#Preparar todos los datos con el Standadizer
std_mach = machine(std_model, X)
fit!(std_mach, verbosity=0)
Xs = transform(std_mach, X)

# Crear los scatterplots en una cuadrícula
plot_layout = @layout [a b; c d]
plots = [plot_transformed_data(name, reducer, Xs, y) for (name, reducer) in techniques]

# Mostrar todos los scatterplots en una cuadrícula
plot(plots..., layout=plot_layout)

## Ejercicio propuesto

Como ejercicio adicional, se propone incorporar otras técnicas de reducción, como **t-SNE**, **Isomap** o **LLE**.  
Aunque algunas de ellas no están integradas directamente en MLJ, pueden utilizarse mediante *wrappers* o implementaciones externas.  
Estas técnicas son especialmente útiles para la **visualización** de datos en espacios de baja dimensión y pueden complementar las vistas obtenidas con los métodos incluidos en este notebook.
